In [7]:
############### EEG dataset #################

# Disable info messages from the tensorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score
from scipy.ndimage import zoom
import pickle

In [20]:
############### Load the dataset ###################
path = 'connect-4.csv'
classes = 3
dataset = np.genfromtxt("connect-4.csv", dtype='str', delimiter=",")
X = dataset[:,0:42]
Y = dataset[:,42]
#Y = LabelEncoder().fit_transform(Y)
#Y = to_categorical(Y,classes)
print(X.shape,Y.shape)

(67557, 42) (67557,)


In [30]:
x = np.zeros(X.shape)

for i, row in enumerate(X):
    for j, col in enumerate(row):
        if col == 'x':
            x[i,j] = 1.0
        if col == 'o':
            x[i,j] = -1.0
        if col == 'b':
            x[i,j] = 0.0

Y = LabelEncoder().fit_transform(Y)
y = to_categorical(Y)
x = MinMaxScaler().fit_transform(x)
print(x.shape,y.shape)

(67557, 42) (67557, 3)


In [31]:
X_train,X_test, Y_train,Y_test = train_test_split(x,y,test_size=0.259884)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(50000, 42) (50000, 3)
(17557, 42) (17557, 3)


In [32]:
x_train,x_test = X_train,X_test
n = X_train.shape[1]
# convert 2-D to 3-D.
for i in range(1,n):
  model = Sequential()
  model.add(Dense(n,name='feature', activation='relu',input_shape=(n,)))
  model.add(Dense(classes,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer='adam')
  model.fit(X_train,Y_train,epochs=1,batch_size=128,verbose=0)
  extract = Model(model.inputs, model.get_layer('feature').output)
  features1 = extract.predict(X_train)
  features2 = extract.predict(X_test)
  x_train = np.concatenate((x_train, features1), axis=1) # concatenate on horizontal axis
  x_test = np.concatenate((x_test, features2), axis=1)
print(x_train.shape,x_test.shape)


(50000, 1764) (17557, 1764)


In [33]:
from PIL import Image
xtrain = []
xtest = []
for i in range(len(x_train)):
    img = Image.fromarray(x_train[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    xtrain.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
xtrain = np.array(xtrain,dtype='float32')
for i in range(len(x_test)):
    img = Image.fromarray(x_test[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    xtest.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
xtest = np.array(xtest,dtype='float32')
print(xtrain.shape,xtest.shape)

(50000, 32, 32, 3) (17557, 32, 32, 3)


In [34]:
############### Feature-extraction ###############
# pretrained model for FE
pretrained_model = keras.applications.DenseNet121(input_shape=(32,32,3), weights=None, include_top=False)
x = Flatten()(pretrained_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(3, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# fit training data and validate on the testing data
model.fit(xtrain,Y_train,batch_size=128, epochs=20,verbose=1,validation_data=(xtest,Y_test))

Epoch 1/20
391/391 [==============================] - 21s 55ms/step - loss: 0.6606 - accuracy: 0.7343 - val_loss: 1.7298 - val_accuracy: 0.2463
Epoch 2/20
391/391 [==============================] - 19s 48ms/step - loss: 0.6738 - accuracy: 0.7336 - val_loss: 0.7489 - val_accuracy: 0.7353
Epoch 3/20
391/391 [==============================] - 19s 48ms/step - loss: 0.6387 - accuracy: 0.7444 - val_loss: 0.7742 - val_accuracy: 0.7149
Epoch 4/20
391/391 [==============================] - 19s 48ms/step - loss: 0.6177 - accuracy: 0.7499 - val_loss: 1.7980 - val_accuracy: 0.4416
Epoch 5/20
391/391 [==============================] - 19s 49ms/step - loss: 0.6061 - accuracy: 0.7526 - val_loss: 0.6527 - val_accuracy: 0.7352
Epoch 6/20
391/391 [==============================] - 19s 49ms/step - loss: 0.6059 - accuracy: 0.7548 - val_loss: 9.8960 - val_accuracy: 0.6922
Epoch 7/20
391/391 [==============================] - 19s 49ms/step - loss: 0.6190 - accuracy: 0.7528 - val_loss: 1.0838 - val_accuracy:

In [36]:
score = model.evaluate(xtest,Y_test)
acc = model.evaluate(xtrain,Y_train)
print('training accuracy: ',acc[1])
print('testing accuracy: ',score[1])

1563/1563 [==============================] - 15s 10ms/step - loss: 0.6116 - accuracy: 0.7480
training accuracy:  0.747979998588562
testing accuracy:  0.7379962205886841
